## Hint: Make a copy of this notebook so you don't need to worry about git conflicts

In [ ]:
import os

import labbox_ephys as le
import labbox_ephys_widgets_jp as lew
import kachery_p2p as kp

# ipykernel is giving some annoying deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Load and view the default workspace

In [ ]:
workspace_uri = kp.get('sortingview-default-workspace')
if not workspace_uri:
    workspace_uri = le.create_workspace(label='sortingview-default').uri
    kp.set('sortingview-default-workspace', workspace_uri)
workspace = le.load_workspace(workspace_uri)
print(f'Workspace URI: {workspace.uri}')

In [ ]:
# View the workspace
workspace_view = lew.WorkspaceView(workspace=workspace)
display(workspace_view)

# Add recordings/sortings to the workspace

Each time you run this cell, new data will be added to the workspace, which will persist in the loaded feed

In [ ]:
import spikeextractors as se
import numpy as np

recording_label = 'simulated_recording'
duration_sec = 50 # duration of simulated recording
num_channels = 8 # num. channels in simulated recording
num_units = 5 # num units
seed = 1 # random number generator seed

def prepare_recording_sorting():
    # Simulate a recording (toy example)
    recording, sorting = se.example_datasets.toy_example(duration=duration_sec, num_channels=num_channels, K=num_units, seed=seed)
    R = le.LabboxEphysRecordingExtractor.from_memory(recording, serialize=True, serialize_dtype=np.int16)
    S = le.LabboxEphysSortingExtractor.from_memory(sorting, serialize=True)
    return R, S

recording, sorting_true = prepare_recording_sorting()
sorting_label = 'true'
R_id = workspace.add_recording(recording=recording, label=recording_label)
S_id = workspace.add_sorting(sorting=sorting_true, recording_id=R_id, label=sorting_label)

# Get info about the currently selected recording/sorting

In the above workspace view, click to open a recording and sorting

In [ ]:
# Get the current selected reocrding and sorting id
recording_id = workspace_view.current_recording_id
sorting_id = workspace_view.current_sorting_id
print('Current recording and sorting from the workspace view above')
print(f'Recording: {recording_id}; Sorting: {sorting_id}')

In [ ]:
# Get the recording extractor and sorting extractor
R = workspace.get_recording_extractor(recording_id)
S = workspace.get_sorting_extractor(sorting_id)
print(f'Channels: {R.get_channel_ids()}')
print(f'Units: {S.get_unit_ids()}')
print(f'Sampling freq. (Hz): {R.get_sampling_frequency()}')

In [ ]:
# Get the sortingcuration and curated sorting extractor
sorting_curation = workspace.get_sorting_curation(sorting_id)
print(sorting_curation)
S_curated = workspace.get_curated_sorting_extractor(sorting_id)
print(f'Units: {S_curated.get_unit_ids()}')

# Set external unit metrics for a sorting

In [ ]:
# set some external unit metrics
# This will be reflected in the units table above
m = [
    {'name': 'test1', 'label': 'Test metric 1', 'tooltip': 'Test metric 1', 'data': {'1': 10, '2': 20, '3': 37}},
    {'name': 'test2', 'label': 'Test metric 2', 'tooltip': 'Test metric 2', 'data': {'1': -1, '2': -2, '3': -3}}
]
workspace.set_unit_metrics_for_sorting(sorting_id=sorting_id, metrics=m)